<a href="https://colab.research.google.com/github/Grismonde/TER_Art/blob/main/Analyse_de_texte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!kill -9 -1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import de spacy, librairies de traitement de texte. Liens utiles : 

*  https://medium.com/@dhifallah.othmen/extraction-et-entra%C3%AEnement-des-entit%C3%A9s-nomm%C3%A9es-avec-spacy-90f7e1a219d7
*  https://course.spacy.io/fr/
*  https://datacorner.fr/spacy/

In [4]:
!pip install -U spacy
!python -m spacy download fr_core_news_md
import spacy
from spacy import displacy

nlp = spacy.load('fr_core_news_md')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.1 which is incompatible.
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-11 20:10:30.051515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enab

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Code qui permet la normalisation du texte

In [5]:
import re
import numpy as np
from bs4 import BeautifulSoup
from collections import Counter

In [9]:
# Charger le fichier HTML
with open("/content/drive/MyDrive/Colab Notebooks/gd-fiscalite_depenses-utf8-sgml-cwb.html", "r") as f:
    html = f.read()

# Supprimer toutes les balises HTML
html = re.sub(r'<[^>]*?>', '', html)

# Écrire le contenu modifié dans un fichier HTML
with open("fichier_modifie.html", "w") as f:
    f.write(html)

# Supprimer les lignes vides du fichier modifié
with open("fichier_modifie.html", "r") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines if line.strip()]

with open("fichier_sans_lignes_vides.html", "w") as f:
    f.write("\n".join(lines))

Affiche le début du contenu du texte "fichier_sans_lignes_vides.html"

In [10]:
from IPython.display import HTML

with open("fichier_sans_lignes_vides.html", "r") as f:
    content = f.readlines()
    
HTML("".join(content[:10]))#Affiche les 10 premieres lignes

Premier traitement de texte avec spacy tokenisation de tout le texte

In [12]:
# Analyser le texte en utilisant SpaCy
with open("fichier_sans_lignes_vides.html", "r", encoding="utf-8") as f:
    # Lecture des 100 premières lignes
    lines = f.readlines()
    # Concaténation des lignes en un seul texte
    text = "".join(lines)
doc = nlp(text)

# Compter la fréquence de chaque mot dans le texte
word_freq = Counter()
for token in doc:
    if not token.is_stop and not token.is_punct and not token.is_space:
        word_freq[token.text] += 1

# Afficher les 50 mots les plus fréquents dans le texte
top_words = word_freq.most_common(20)
for word, freq in top_words:
    print(word, freq)

ValueError: ignored

In [15]:
def process_doc(doc):
    word_freq = Counter()
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            word_freq[token.text] += 1
    return word_freq

# Ouvrir le fichier et récupérer son contenu
with open("fichier_sans_lignes_vides.html", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Nombre de lignes à traiter à la fois
chunk_size = 1000

# Parcourir le texte en segments de taille chunk_size
word_freq = Counter()
for i in range(0, len(lines), chunk_size):
    chunk = "".join(lines[i:i+chunk_size])
    doc = nlp(chunk)
    chunk_word_freq = process_doc(doc)
    word_freq += chunk_word_freq

# Afficher les 50 mots les plus fréquents dans le texte
top_words = word_freq.most_common(20)
for word, freq in top_words:
    print(word, freq)


aides 207796
impôt 164713
impôts 161436
conditions 114205
sociales 112443
revenus 111177
faut 110940
attribution 109625
revenu 101839
dépenses 90100
faire 88713
TVA 81948
travail 81532
personnes 81005
temps 77121
payer 76615
France 72416
Revoir 70148
entreprises 69981
santé 66762


Pour le moment nous allons continuer avec des exemples simples

In [ ]:
doc = nlp("Terrible désillusion pour la championne du monde")
for chunk in doc.noun_chunks:
    print(chunk.text, " --> ", chunk.label_)

Terrible désillusion  -->  NP
la championne  -->  NP
monde  -->  NP


In [ ]:
doc = nlp("Demain je travaille en France chez Microsoft")
for ent in doc.ents:
    print(ent.text, ent.label_)

France LOC
Microsoft ORG
